In [ ]:
import re
import pandas as pd
import os             
from collections import Counter


def extract_the_shot(line):
    reg = re.search("\s*(CU)\s+.*|\s*(C.U.)\s+.*|\s*(WIDE)\s+.*|\s*(POV)\s+.*|\s*(FM)\s+.*|\s*(SM)\s+.*|",line)
    if reg:
        return reg.group()

def extract_num_of_shots(folder_input):
    words = ["CU","WIDE","C.U.","POV", "FM","SM"]
    film_here= []
    lines_reg = []
    film_shots = {}
    new_film_shot = {}
    d_film_shot = {}
    the_list_with_shots =[]
    data_of_filmshots = []
    folder = os.listdir(folder_input)
    for filename in folder:
        if '.txt' in filename:
            with open(folder_input+filename, encoding="utf8", errors='ignore') as f:
                for line in f:
                    shot= extract_the_shot(line)
                    if shot:
                        if filename not in film_shots:
                            film_shots[filename]= []
                        film_shots[filename].append(shot)
                for key,value in film_shots.items():
                    film_here.append(key)
                    lines_reg = list(value)
                for element in lines_reg:
                    for i in words:
                        if i in element:
                            if filename not in new_film_shot:
                                new_film_shot[filename] = []
                            new_film_shot[filename].append(i)
        for key,value in new_film_shot.items():
            new_film_shot[key]=Counter(value)
            d_film_shot = new_film_shot
    for key, value in d_film_shot.items():
        the_list_with_shots.append(key.replace('.txt',"").split(";"))
        data_of_filmshots.append(value)
    film_data = pd.DataFrame(the_list_with_shots, columns = ["year", "genre", "title"])
    shot_data = pd.DataFrame(data_of_filmshots)
    data_f= pd.concat([film_data,shot_data],axis=1)
    return data_f 


def extract_shot(line):
    r_shot = re.search('CLOSEUP|CLOSE UP|WIDE VIEW|LONG SHOT|MEDIUM|MEDIUM SHOT|'
                    'MEDIUM SHOT.|MEDIUM CLOSE SHOT|MEDIUM CLOSEUP|MED.CLOSE SHOT|'
                    'MEDIUM LONG SHOT|MED. LONG SHOT|EXTREME CLOSE UP|EXTREME CLOSE|'
                    'EXTREME CLOSEUPS|PAN RIGHT|PAN LEFT|PAN UP|PAN DOWN|TILT UP|'
                    'TILT DOWN|TILT RIGHT|TILT LEFT|POINT OF VIEW|SLOW MOTION|FAST MOTION|'
                    'SLO-MO|ZOOM OUT|ZOOM BACK|ZOOM IN',line) 
    if r_shot:
        return r_shot.group()

def extract_number_of_shots(folder_input):
    film_shots = {}
    d_film_shot = {}
    the_list_with_title =[]
    data_of_filmshots = []
    folder = os.listdir(folder_input)
    for filename in folder:
        if '.txt' in filename:
            with open(folder_input+filename, encoding="utf8", errors='ignore') as f:
                for line in f:
                    shot= extract_shot(line)
                    if shot:
                        if filename not in film_shots:
                            film_shots[filename]= []
                        film_shots[filename].append(shot)
        for key,value in film_shots.items():
            film_shots[key]=Counter(value)
            d_film_shot = film_shots
    for key, value in d_film_shot.items():
        the_list_with_title.append(key.replace('.txt',"").split(";"))
        data_of_filmshots.append(value)
    film_data = pd.DataFrame(the_list_with_title, columns = ["year", "genre", "title"])
    shot_data = pd.DataFrame(data_of_filmshots)
    df= pd.concat([film_data,shot_data],axis="columns")
    return df

def add_file():
    folder_input =''
    folder_output =''
    first_file = extract_num_of_shots(folder_input)
    second_file = extract_number_of_shots(folder_input)
    new = second_file.merge(first_file, on= ("genre","title","year"))
    new.to_csv(folder_output+'_number_of_shots.csv')
